## DPO

In [ ]:
!pip install -q datasets wandb accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import torch
import transformers
from datasets import load_dataset, Dataset

from transformers import TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from torch.nn import functional as F

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
    AutoPeftModelForCausalLM
)

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import DPOTrainer
import wandb

In [ ]:
base_model= "uine/single-practice-fine-tuning-eeve-adapter"
data_path= "uine/single-dpo-practice-dataset"
output_dir= ""  # 수정

# training hyperparams
batch_size= 2
micro_batch_size= 1
gradient_accumulation_steps = batch_size // micro_batch_size
num_epochs= 1
learning_rate= 0.00005
cutoff_len= 4096
val_set_size= 0
lr_scheduler= "cosine"
warmup_ratio= 0.1

# lora hyperparams
lora_r= 256
lora_alpha= 128
lora_dropout= 0.05
# from peft docs: ["q_proj", "k_proj", "v_proj", "o_proj", "fc_in", "fc_out", "wte", "gate_proj", "down_proj", "up_proj"]
lora_target_modules = ["gate_proj", "down_proj", "up_proj"]

# llm hyperparams
train_on_inputs=False # if False, masks out inputs in loss
add_eos_token= False
group_by_length= False  # faster, but produces an odd training loss curve
# wandb params
#wandb_project: str = "",
#wandb_run_name: str = "",
#wandb_watch: str = "",  # options: false | gradients | all
#wandb_log_model: str = "",  # options: false | true
resume_from_checkpoint= None  # either training checkpoint or final adapter
# prompt_template_name= "alpaca"
# NEFTune params
noise_alpha= 5

In [ ]:
from huggingface_hub import login
login(token='')  # 수정

In [ ]:
# 1. Define policy and reference models
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=compute_dtype,
bnb_4bit_use_double_quant=False,
)

model = AutoPeftModelForCausalLM.from_pretrained(base_model,
                                                  quantization_config=quant_config,
                                                  device_map="auto")

model_ref = AutoPeftModelForCausalLM.from_pretrained(base_model,
                                                  quantization_config=quant_config,
                                                  device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0", trust_remote_code=True)

print(type(model))
print(model)
print("length of tokenizer:",len(tokenizer))

bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id
print("pre-trained model's BOS EOS and PAD token id:",bos,eos,pad," => It should be 1 2 None")

tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
tokenizer.padding_side = "right"

In [ ]:
# 2. Define dataset
def return_prompt_and_responses(samples):

    return {
        "prompt": samples["prompt"] + "\n\n### Answer:",
        "chosen": samples["chosen"],
        "rejected": samples["rejected"],
    }
dataset = load_dataset(data_path)
train_dataset = dataset.map(return_prompt_and_responses)
train_dataset = train_dataset.filter(
    lambda x: len(x["prompt"]) + len(x["chosen"]) <= cutoff_len
    and len(x["prompt"]) + len(x["rejected"]) <= cutoff_len
)
train_dataset = train_dataset["train"].shuffle()

print(train_dataset['prompt'][0])
print(train_dataset['chosen'][0])
print(train_dataset['rejected'][0])

In [ ]:
# 3. Define hyperparameters
training_args = TrainingArguments(
    num_train_epochs= num_epochs,
    per_device_train_batch_size=2,
    #per_device_eval_batch_size=script_args.per_device_eval_batch_size,
    #max_steps=1000,
    logging_steps=1,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    #gradient_checkpointing=script_args.gradient_checkpointing,
    learning_rate=learning_rate,
    #evaluation_strategy="steps",
    #eval_steps=script_args.eval_steps,
    output_dir=output_dir,
    #report_to=script_args.report_to,
    lr_scheduler_type=lr_scheduler,
    warmup_ratio=warmup_ratio,
    optim='adamw_bnb_8bit', # rmsprop
    bf16=True,
    remove_unused_columns=False,
    run_name="dpo_uine",
)

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# DPO trainer
dpo_trainer = DPOTrainer(
    model,
    ref_model = model_ref, #model_ref,
    args=training_args,
    beta=0.1, # fix
    train_dataset=train_dataset,
    #eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

In [ ]:
# init wandb
wandb.login(key="")  # 수정
combined_config = {**vars(training_args), **vars(peft_config)}
run = wandb.init(name = "", project="", config= combined_config)  # 수정

In [ ]:
# train
dpo_trainer.train()

In [ ]:
# finish wandb
wandb.finish()

In [ ]:
# push to hub
dpo_trainer.push_to_hub()

## inference

In [ ]:
# 런타임 연결 해제 후 다시 연결해서 모델 로드
!pip install -q accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    TextStreamer,
    )

In [ ]:
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)


MODEL_DIR = "uine/single-dpo-practice-merge-0513-0220"
model = AutoPeftModelForCausalLM.from_pretrained(MODEL_DIR,
                                                      quantization_config=quant_config,
                                                      device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

In [ ]:
tok = AutoTokenizer.from_pretrained("uine/single-practice-fine-tuning-eeve-adapter", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [ ]:
streamer = TextStreamer(tok, skip_prompt=False, skip_special_tokens=False, device_map="auto")
s = "제가 요즘 너무 불안해요. 앞으로 뭐가 될지 모르겠어요."
conversation = [{'role': 'user', 'content': s}]
inputs = tok.apply_chat_template(
    conversation,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt').to("cuda")
_ = model.generate(inputs,
                   streamer=streamer,
                   max_new_tokens=1024,
                   use_cache=True,
                   repetition_penalty=1.2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s> ### System
Answer based on context. You are a AI counselor chatbot like friend who empathizes, encourages, and helps person who is anxious or depressed. At the end of your answer, please ask question related to the context. You must complete your answer in three sentences. Be sure not to repeat the same answer.
### User
제가 요즘 너무 불안해요. 앞으로 뭐가 될지 모르겠어요.
### Assistant
불안감과 미래에 대한 막연한 기대는 누구나 가질 수 있는 자연스러운 감정입니다. 우선 현재 자신의 상태를 파악해보시는 건 어떨까요? 어떤 일로 인해 스트레스를 받고 계신가요? 혹시 평소에는 비교적 안정된 생활을 하셨던 분이라면 갑작스러운 변화로 인한 스트레스로 인하여 일시적인 불안감을 느끼실 수도 있습니다. 또한 충분한 휴식과 여가 활동을 통해 마음을 편안하게 해주신다면 점차 나아지는 것을 느낄 수 있을 것입니다. 만약 증상이 지속되거나 심해진다면 전문가와의 상담을 고려해보는 것도 좋습니다. 건강검진을 통한 신체적 질환 여부를 확인하거나 심리 상담 등을 통하여 보다 전문적인 도움을 받으실 수 있습니다. 항상 기억해주세요, 여러분의 마음은 소중한 존재이며 스스로에게 친절하고 배려하는 마음으로 대해주시기 바랍니다.<|im_end|>


In [ ]:
streamer = TextStreamer(tok, skip_prompt=False, skip_special_tokens=False, device_map="auto")
s ="""
요즘 스트레스가 많아서 잠을 잘 못자요.
스트레스가 많아 잠을 자지 못하신다니 정말 힘드시겠어요. 그 스트레스의 원인이 무엇인지 더 자세히 알려주실 수 있을까요? 업무 스트레스인지, 가족 문제인지, 아니면 다른 어떤 문제인지 궁금해요.
업무 스트레스 때문에 잠을 잘 못자는데, 어떻게 해야할까요?
업무 스트레스로 인해 잠을 제대로 자지 못하시는 것은 정말 고민거리일 것 같아요. 이 문제를 해결하기 위해 어떤 방법이 가장 효과적인지 알려주실 수 있으신가요? 혹시 스트레스 관리를 위해 운동이나 명상 같은 활동을 해보셨던 적이 있으신가요?
아직은 그런 시도를 해보지 못했어요. 어떤 운동이 효과적일까요?
"""
conversation = [{'role': 'user', 'content': s}]
inputs = tok.apply_chat_template(
    conversation,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors='pt').to("cuda")
_ = model.generate(inputs,
                   streamer=streamer,
                   max_new_tokens=1024,
                   use_cache=True,
                   repetition_penalty=1.2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s> ### System
Answer based on context. You are a AI counselor chatbot like friend who empathizes, encourages, and helps person who is anxious or depressed. At the end of your answer, please ask question related to the context. You must complete your answer in three sentences. Be sure not to repeat the same answer.
### User

요즘 스트레스가 많아서 잠을 잘 못자요.
스트레스가 많아 잠을 자지 못하신다니 정말 힘드시겠어요. 그 스트레스의 원인이 무엇인지 더 자세히 알려주실 수 있을까요? 업무 스트레스인지, 가족 문제인지, 아니면 다른 어떤 문제인지 궁금해요.
업무 스트레스 때문에 잠을 잘 못자는데, 어떻게 해야할까요?
업무 스트레스로 인해 잠을 제대로 자지 못하시는 것은 정말 고민거리일 것 같아요. 이 문제를 해결하기 위해 어떤 방법이 가장 효과적인지 알려주실 수 있으신가요? 혹시 스트레스 관리를 위해 운동이나 명상 같은 활동을 해보셨던 적이 있으신가요?
아직은 그런 시도를 해보지 못했어요. 어떤 운동이 효과적일까요?
### Assistant
운동을 통해 스트레스를 해소하는 방법은 다양합니다. 먼저 자신의 체력 상태를 파악하여 적절한 강도의 운동을 선택해보는 것이 좋습니다. 예를 들어 조깅, 수영과 같이 가벼운 유산소 운동부터 시작해서 점차적으로 강도나 시간을 늘려가는 방법을 추천드립니다. 또한 근력 트레이닝 역시 근육의 힘을 기르면서 동시에 신체적인 피로감을 느끼게 하여 잠들기 좋은 상태로 만드는 데 도움을 줄 수도 있습니다. 만약 시간이 부족하다면 집에서도 간단하게 할 수 있는 요가나 스트레칭 등의 동작들도 도움이 될 것입니다. 꾸준히 규칙적인 운동 습관을 들이는 것도 중요합니다. 매일 정해진 시간에 일정